# Segmentação dos usuários com base em suas características

### Por nicho de mercado:

In [33]:
import pandas as pd
import numpy as np

In [34]:
df = pd.read_csv('data_source_hotmart.csv')
df.purchase_value = df.purchase_value.str.replace(',', '.').astype(float)
df.head()

,purchase_id,product_id,affiliate_id,producer_id,buyer_id,purchase_date,product_creation_date,product_category,product_niche,purchase_value,affiliate_commission_percentual,purchase_device,purchase_origin,is_origin_page_social_network
0,1663958,6640,209372,116238,1200397,2016-06-26 12:00:00,2011-03-19 15:47:36,Video,Presentation skills,180.61,"0,5",Smart TV,Origin ef2b,"0,0"
1,1677087,2350,141418,2821,1083764,2016-06-26 12:00:00,2010-07-05 01:50:15,Podcast,Child psychology,220.32,"0,6",Smart TV,Origin ef2b,"0,0"
2,2017360,35669,618642,618642,1436106,2016-06-26 12:00:00,2012-06-13 02:59:37,Podcast,Presentation skills,88.95,"0,0",Smart TV,Origin ef2b,"0,0"
3,2017379,57998,1164511,70388,1436118,2016-06-26 12:00:00,2013-05-07 08:51:31,Podcast,Anxiety management,119.47,"0,5",Smart TV,Origin ef2b,"0,0"
4,2017382,58329,1261488,221253,1386357,2016-06-26 12:00:00,2013-05-12 08:12:06,Podcast,Teaching English,96.38,"0,5",Smart TV,Origin ef2b,"0,0"


### Com base no nicho de mercado dos produtos, busquei encontrar usuários que compram produtos somente de um mesmo nicho

Agrupando os dados por usuário e nicho de mercado temos a quantidade de produtos de cada nicho comprado por cada um dos usuários:

In [35]:
buyer_id_Groups = df[['buyer_id', 'product_category', 'product_niche']].groupby(['buyer_id', 'product_niche']).count().reset_index()
buyer_id_Groups.columns = ['buyer_id', 'product_niche', 'qnt']
buyer_id_Groups.head()

,buyer_id,product_niche,qnt
0,60,Procrastination,2
1,65,Anxiety management,6
2,80,Presentation skills,1
3,91,YouTube video creation,4
4,115,Presentation skills,1


Em seguida filtrei usuários que realizaram compra apenas de um único nicho, e mais de uma vez, pra obter uma segmentação mais limpa:

In [36]:
Group_Niche = buyer_id_Groups[~buyer_id_Groups.buyer_id.duplicated()]
Group_Niche = Group_Niche[Group_Niche.qnt > 1]
Group_Niche.head()

,buyer_id,product_niche,qnt
0,60,Procrastination,2
1,65,Anxiety management,6
3,91,YouTube video creation,4
6,136,YouTube video creation,5
14,287,Media training,2


Aqui vejo a quantidade de usuários por nicho que podem ser segmentados apenas por esse critério:

In [37]:
print('Usuários na base:', df.buyer_id.nunique())
print('Usuários segmentados somente pelo nicho de mercado do produto:', Group_Niche.buyer_id.count(), "\n")

Group_Niche_csv = pd.DataFrame(Group_Niche.product_niche.value_counts()).reset_index()
Group_Niche_csv.columns = ['Nicho de Mercado', 'Qnt_usuários']
Group_Niche_csv.head()

Usuários na base: 1100649
Usuários segmentados somente pelo nicho de mercado do produto: 157854 



,Nicho de Mercado,Qnt_usuários
0,Negotiation,28017
1,Anxiety management,22179
2,Personal finance,21179
3,Immigration,20484
4,YouTube video creation,9707


In [38]:
Group_Niche_csv.to_csv('Group_Niche_hotmart.csv', header=True, index=False, encoding='UTF-8')

### Por categoria do produto:

Filtrei o restante da base de usuários que não foram segmentados pelo nicho de mercado e utilizei a mesma técnica de segmentação anterior, só que agora pra categoria do produto.

In [39]:
Group_Category = df[~df.buyer_id.isin(Group_Niche.buyer_id)][['buyer_id', 'product_category', 'product_id']].groupby(['buyer_id', 'product_category']).count().reset_index()
Group_Category.columns = ['buyer_id', 'product_category', 'qnt']
Group_Category = Group_Category[Group_Category.qnt>1]
Group_Category.head()

,buyer_id,product_category,qnt
12,310,Phisical book,3
16,354,Phisical book,2
18,372,Phisical book,2
20,424,Phisical book,2
21,427,Phisical book,2


In [40]:
print('Usuários restantes na base:', df[~df.buyer_id.isin(Group_Niche.buyer_id)].buyer_id.nunique())
print('Usuários segmentados somente pelo nicho de mercado do produto:', Group_Category.buyer_id.count(), "\n")

Group_Category_csv = pd.DataFrame(Group_Category.product_category.value_counts()).reset_index()
Group_Category_csv.columns = ['Categoria do Produto', 'Qnt_usuários']
Group_Category_csv.head()

Usuários restantes na base: 942795
Usuários segmentados somente pelo nicho de mercado do produto: 57193 



,Categoria do Produto,Qnt_usuários
0,Phisical book,52040
1,Podcast,4040
2,Workshop,1070
3,eBook,13
4,Webinar,11


In [41]:
Group_Category_csv.to_csv('Group_Category_hotmart.csv', header=True, index=False, encoding='UTF-8')

### Por faixa de preço médio dos produtos comprados:

Outra maneira de segmentar os usuários é pelo valor médio gasto por eles, então criei faixas de valores para segmentar o restante da base. 

In [42]:
purchase_value = df[~(df.buyer_id.isin(Group_Niche.buyer_id)) & ~(df.buyer_id.isin(Group_Category.buyer_id))]
purchase_value = purchase_value[['buyer_id', 'purchase_value']].groupby(['buyer_id']).mean().reset_index()
purchase_value.columns = ['buyer_id', 'avg_purchase_value']
purchase_value.head()

,buyer_id,avg_purchase_value
0,80,265.09
1,115,73.25
2,120,74.18
3,139,154.01
4,169,391.79


In [43]:
cuts = np.arange(purchase_value.avg_purchase_value.min(), purchase_value.avg_purchase_value.max(), 200)
purchase_value['avg_purchase_bins'] = pd.cut(purchase_value['avg_purchase_value'], cuts)

Quantidade de usuários por faixa de valor médio gasto:

In [44]:
Group_purchase = pd.DataFrame(purchase_value.avg_purchase_bins.value_counts()).reset_index()
Group_purchase.columns = ['avg_purchase_bins', 'qnt_users']
Group_purchase = Group_purchase[Group_purchase.qnt_users>0]
Group_purchase.head()

,avg_purchase_bins,qnt_users
0,"(56.44, 256.44]",551102
1,"(256.44, 456.44]",156888
2,"(456.44, 656.44]",82603
3,"(856.44, 1056.44]",25938
4,"(656.44, 856.44]",19068


In [45]:
Group_purchase.to_csv('Group_purchase_hotmart.csv', header=True, index=False, encoding='UTF-8')